# Variational Quantum Eigensolver (VQE)

A variational quantum eigensolver is a variational quantum algorithm where a parametrized quantum circuit is trained to prepare the ground state of a target Hamiltonian [See A. Peruzzo et al. - 2013](https://arxiv.org/abs/1304.3061).

<img src="./figures/vqe.png" width="800" height="500" alt="My Image">

As sketched above, the idea is that we get a state from a quantum circuit, and this state depends on the parameters of the circuit. Then we implement a machine learning routine to update the parameters of the circuit such that the expectation value of our target Hamiltonian on this state is minimized.

<div style="background-color: rgba(255, 105, 105, 0.3); border: 2.5px solid #000000; padding: 15px;">
    <strong>Exercise:</strong> 
    Solve the Grover problem using a VQE. Suppose that you want to find the $i$-th item in a database of $2^n$ items. To do that you have to formulate the problem as an energy minimization problem: you first define an Hamiltonian which encodes in its ground state the $i$-th item you are interested in, then you train your QML model to miminize the expectation value of such Hamiltonian.
    
</div>

### Problem's Hamiltonian
First you'll have to come up with the Hamiltonian form that solves the problem, i.e. that encodes the i-th item in its ground state. 
You can use ``qibo``'s ``SymbolicHamiltonian`` and symbols to define the form of your Grover's Hamiltonian.

In [1]:
from qibo.hamiltonians import SymbolicHamiltonian
from qibo.symbols import Z, I

# this creates a Z0 * I1 operator, i.e. a tensor product between a pauli Z
# on the first qubit and the identity on the second qubit
form = Z(0) * I(1)
# you can also add other terms
form += 1 - Z(2) * (1 + I(0) * Z(1) * Z(2))
form

[Qibo 0.2.20|INFO|2025-07-17 17:56:41]: Using qibojit (cupy) backend on /GPU:0


1 + Z0*I1 - Z2*(1 + I0*Z1*Z2)

**hint 1:** 

<details>
<summary> show </summary>
If you represent items in the database as bitstrings, then supposing the item you're looking for corresponds to the state with $| x^\ast \rangle = | x_0x_1 ... x_n \rangle$, you may want to write the projector onto that state.
</details>

**hint 2:** 

<details>
<summary> show </summary>
The projector you need is therefore $| x^\ast \rangle \langle x^\ast|$ and you can build the hamiltonian as: 
    $$H = 1 - | x^\ast \rangle \langle x^\ast |$$ 
which, since $x^\ast$ is a string of bits, can be written in terms of $Z$ pauli operators as:
    $$H = 1 - \bigotimes_i \bigg( 1 + (-1)^{x_i} Z_i \bigg)$$
where the $x_i$ are the bits composing $x^\ast$.
</details>

In [5]:
import numpy as np

def grover_hamiltonian(item_index, n_items):
    # extract how many qubits you need
    nqubits = (n_items - 1).bit_length()
    # build the binary representation
    bitstring = f"{item_index:0{nqubits}b}"
    bits = [int(b) for b in bitstring]
    # projector on the desired state
    form = 1 - np.prod([(1 + (-1)**bits[q] * Z(q) ) for q in range(nqubits)])
    return SymbolicHamiltonian(form)

item_index = 11
n_items = 32

H = grover_hamiltonian(item_index, n_items)
H.form

1 - (1 + Z0)*(1 - Z1)*(1 + Z2)*(1 - Z3)*(1 - Z4)

### Build the QML model

Now that we have the Hamiltonian, you can build the desired QML model using the tools introduced in the previous notebook. 

In [8]:
from qiboml.models.ansatze import HardwareEfficient
from qiboml.models.decoding import Expectation
from qiboml.interfaces.pytorch import QuantumModel

from qibo import set_backend, gates

import torch

set_backend("qiboml", platform="pytorch")

nqubits = 5
item_index = 11
n_items = 32

# parametrized circuit
trainable_circuit = HardwareEfficient(nqubits=nqubits, nlayers=3)
# decoder
decoder = Expectation(
    nqubits=nqubits,
    observable=grover_hamiltonian(item_index, n_items), # this is the default choice anyway
)
model = QuantumModel(
    circuit_structure=[trainable_circuit,],
    decoding=decoder,
)

dev = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")
model = model.to(dev)

optimizer = torch.optim.Adam(model.parameters(), lr=0.1)

for epoch in range(20):
    optimizer.zero_grad()
    cost = model()
    cost.backward()
    optimizer.step()    
    print(f"Epoch {epoch}: Energy = {cost.item()}")
    
circ = trainable_circuit.copy(True)
circ.add(gates.M(*range(nqubits)))

print(f"{item_index:0{nqubits}b}")
print(circ().frequencies())

[Qibo 0.2.20|INFO|2025-07-17 18:03:00]: Using qiboml (pytorch) backend on cuda:0


Epoch 0: Energy = 0.39454764974900497
Epoch 1: Energy = -0.9398785123709582
Epoch 2: Energy = -2.664277760214087
Epoch 3: Energy = -4.782827792936922
Epoch 4: Energy = -7.164139469241632
Epoch 5: Energy = -9.42084905102319
Epoch 6: Energy = -11.281355752310192
Epoch 7: Energy = -12.879771236016632
Epoch 8: Energy = -14.395945224218336
Epoch 9: Energy = -15.93934500051856
Epoch 10: Energy = -17.58934666981554
Epoch 11: Energy = -19.424308457996347
Epoch 12: Energy = -21.34991918995205
Epoch 13: Energy = -23.236498787106466
Epoch 14: Energy = -25.036076015071178
Epoch 15: Energy = -26.675899800942563
Epoch 16: Energy = -28.014848489915252
Epoch 17: Energy = -28.967138828424797
Epoch 18: Energy = -29.54206001913085
Epoch 19: Energy = -29.822521944054937
01011
Counter({'01011': 969, '11010': 9, '00110': 5, '00000': 4, '01001': 3, '01101': 3, '01010': 2, '00101': 1, '00111': 1, '01111': 1, '10111': 1, '11000': 1})
